In [1]:
import pandas as pd
import os
import torch
from transformers import Trainer, BertTokenizer
import numpy as np

In [2]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/GrammatiktakDatasets/otherDatasets/")
test_sentences = pd.read_csv("nutids_r.csv")

In [ ]:
import pickle

os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, batch_size=16):
        self.encodings = encodings
        self.labels = labels
        self.batch_size = batch_size

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score, recall_score, precision_score

char = "*@;:!\"?«».,"

def init_model(model):
    device = "mps"
    torch.device(device)
    model.eval()
    model.to(device)
    return model

def load_constants():
    SCOPE = 6
    PADDING = int(SCOPE/2-1)
    HALF_SCOPE = int(SCOPE/2)
    MAX_LENGTH = 21
    BATCH_SIZE
    return SCOPE, PADDING, HALF_SCOPE, MAX_LENGTH, BATCH_SIZE

class correct_nutidsr():
    def __init__(self):
        os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/FineTuneModels/Models/")
        self.tokenizer = BertTokenizer.from_pretrained('Maltehb/danish-bert-botxo')
        self.model = init_model(torch.load("nutidsrModel1", map_location=torch.device('cpu')))
        self.scope, self.padding, self.half_scope, self.max_length, self.batch_size = load_constants()
    
    def add_padding(self, words):
        return ["<PAD>"]*self.padding + words + ["<PAD>"]*self.padding
    
    def clean_input(self, pre_cleaned_result):
        post_cleaned_result = []
        for i in tqdm(range(len(pre_cleaned_result))):
            pre_cleaned_lst = pre_cleaned_result[i]
            lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
            post_cleaned_result.append(lst)
        return post_cleaned_result
    
    def convert_sentences_to_dataset(self, post_cleaned_result):
        dataset = []
        output_lst = []
        for i in tqdm(range(len(post_cleaned_result))):
            current_lst = post_cleaned_result[i]
            for x in range(len(current_lst)):
                current_word = current_lst[x]
                try: stemmed_word = nutids_r_stem[current_word]
                except: continue
                current_dataset = current_lst[x-self.half_scope:x+self.half_scope+1]
                current_dataset[self.half_scope] = stemmed_word
                dataset.append(" ".join(current_dataset))
                if nutids_r[stemmed_word][0] == current_word:
                    output = 0
                elif nutids_r[stemmed_word][1] == current_word:
                    output = 1
                else:
                    print("ERROR")
                output_lst.append(output)
        return dataset, output_lst

    def convert_dataset_to_dataloader(self, dataset, output):
        test_dataset = CustomDataset(dataset, output, batch_size=self.batch_size)
        return test_dataset
    
    def tokenize_sentences(self, sentences):
        X_tokenized = self.tokenizer(sentences, padding=True, truncation=True, max_length=self.max_length)
        return X_tokenized
    
    def get_predictions(self):
        pass

    def correct_sentence(self):
        pass


